In [1]:
import pandas as pd
import ast

In [19]:
def clean_string(x):
    return x.replace("\t"," ").replace("\n"," ")

q = pd.read_csv("/scratch/s190619/Data_etc/MedQA/disorders_table_dev-test.csv")
q["options"] = q["options"].apply(lambda x: ast.literal_eval(x))
opts = []
for i in range(q["options"].shape[0]):
    opt_tmp = [x for x in q["options"].iloc[i].values() if x != q["answer"].iloc[i]]
    opt_tmp.insert(0, q["answer"].iloc[i])
    opts.append(opt_tmp)
q["options"] = opts 
q = q[["qid","options"]]
for i in range(5):
    q[f"options{i}"] = [clean_string(q.iloc[j]["options"][i]) for j in range(q.shape[0])]
q = q.drop("options",axis=1)
q.head()

,qid,options0,options1,options2,options3,options4
0,0,Cyclic vomiting syndrome,Gastroenteritis,Hypertrophic pyloric stenosis,Gastroesophageal reflux disease,Acute intermittent porphyria
1,1,Hypoperfusion,Hyperglycemia,Metabolic acidosis,Hypokalemia,Hypophosphatemia
2,2,Iron deficiency,Vitamin B12 deficiency,Folate deficiency,Infiltrative bone marrow process,Intravascular hemolysis
3,3,Monomorphic ventricular tachycardia,Premature ventricular contractions,Mitral regurgitation,Third-degree heart block,Acute pericarditis
4,4,Oppositional defiant disorder,Reactive attachment disorder,Conduct disorder,Antisocial personality disorder,Attention-deficit hyperactivity disorder


In [20]:
r = pd.read_csv("/scratch/s190619/Data_etc/ColBERT/retrievals/dev-test/Ranking_Model4_1.tsv", 
                sep="\t", header = None, names = ["qid","pid","rank"])
r.head()

,qid,pid,rank
0,0,1724,1
1,0,16332,2
2,0,2624,3
3,0,17037,4
4,0,19342,5


In [21]:
FZ_passages_path = "/scratch/s190619/Data_etc/FindZebra/collection_FZ_w_titles.tsv"
fz = pd.read_csv(FZ_passages_path, encoding="utf8", sep="\t", header=None, names=["pid","title","passage"])
fz["passage"] = fz["passage"].apply(lambda x: x.replace("\t"," ").replace("\n", " "))
fz.head()

,pid,title,passage
0,0,Kernicterus,Kernicterusrefers to brain damage that may occ...
1,1,47 XXX syndrome,"47 XXX syndrome, also called trisomy X or trip..."
2,2,Pseudoxanthoma elasticum,"Pseudoxanthoma elasticum, PXE, is an inherited..."
3,3,Smith-Magenis Syndrome,SummaryClinical characteristics.Smith-Magenis ...
4,4,Smith-Lemli-Opitz Syndrome,SummaryClinical characteristics.Smith-Lemli-Op...


In [22]:
#retrieval passages
rp = r.join(fz[["pid","passage"]].set_index("pid"), on="pid").join(q.set_index("qid"), on="qid").drop("pid",axis=1)

rp.tail(10)

,qid,rank,passage,options0,options1,options2,options3,options4
610990,610,991,Bogart–Bacall syndrome (BBS) is a voice disord...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610991,610,992,Rheumatoid pleuritisSpecialtyRheumatologyCause...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610992,610,993,Clinical FeaturesAbou Jamra et al. (2011) repo...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610993,610,994,Clinical FeaturesStauffer and Merrihew (1944) ...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610994,610,995,Fire breather’s pneumoniaOther namesFire breat...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610995,610,996,A number sign (#) is used with this entry beca...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610996,610,997,DescriptionPseudohypoparathyroidism (PHP) is a...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610997,610,998,Menkes syndrome is a disorder that affects cop...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610998,610,999,Cornea verticillataOther namesFleischer vortex...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610999,610,1000,Clinical FeaturesNadol and Burgess (1982) desc...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure


In [30]:
save_path = "/scratch/s190619/Data_etc/ColBERT/MC_eval_data/" + "Model4_1_dev-test.tsv"
rp.to_csv(save_path, sep = "\t", index = False, header = False)

In [10]:
# test = pd.read_csv(save_path, sep = "\t", header = None)

In [25]:
rp.tail()

,qid,rank,passage,options0,options1,options2,options3,options4
610995,610,996,A number sign (#) is used with this entry beca...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610996,610,997,DescriptionPseudohypoparathyroidism (PHP) is a...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610997,610,998,Menkes syndrome is a disorder that affects cop...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610998,610,999,Cornea verticillataOther namesFleischer vortex...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure
610999,610,1000,Clinical FeaturesNadol and Burgess (1982) desc...,Chronic obstructive pulmonary disease (COPD),Asthma,Lymphangioleiomyomatosis,Bronchiectasis,Heart failure


In [24]:
i = 0
with open(save_path, 'r') as f:
    for line in f:
        i +=1
        _, _, p, t, o1, o2, o3, o4 = line.strip().split('\t')

In [15]:
i

35002

In [17]:
test.iloc[i-3:i+3]

,0,1,2,3,4,5,6,7
34999,34,1000,Outdated medical termGay bowel syndrome is a t...,Cardiovascular collapse,Internal bleeding,Respiratory depression,Hypothermia,Self-inflicted violence
35000,35,1,Avascular necrosis of the femoral head (ANFH) ...,Fat embolism,Traumatic rhabdomyolysis,Thyrotoxic crisis,Malignant hyperthermia,"Meningitis\n"""
35001,35,2,Beighton (1981) reported a kindred in which 20...,Fat embolism,Traumatic rhabdomyolysis,Thyrotoxic crisis,Malignant hyperthermia,"Meningitis\n"""
35002,35,3,A number sign (#) is used with this entry beca...,Fat embolism,Traumatic rhabdomyolysis,Thyrotoxic crisis,Malignant hyperthermia,"Meningitis\n"""
35003,35,4,A number sign (#) is used with this entry beca...,Fat embolism,Traumatic rhabdomyolysis,Thyrotoxic crisis,Malignant hyperthermia,"Meningitis\n"""
35004,35,5,A number sign (#) is used with this entry beca...,Fat embolism,Traumatic rhabdomyolysis,Thyrotoxic crisis,Malignant hyperthermia,"Meningitis\n"""


In [18]:
line

'"""\n'

In [29]:
(610999/35002)*15

261.84175189989145